In [2]:
import paramiko
from mutagen.mp3 import MP3
from pydub import AudioSegment
import os

def extract_file_prefix(url):
    filename = url.split('/')[-1]
    return filename.split('_')[0]

def first_suffix_for_sound_directory(url):
    filename = url.split('/')[-2]
    return filename

hostname = 'files.clarksonmsda.org'
port = 2022
username = 'kadaliu@clarkson.edu'
password = 'Act.324fgdg'
remote_directory = '/srv/sftpgo/data/users/kadaliu@clarkson.edu/traffic/snd/'
local_image_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultimages/'
local_audio_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultaudio/'

def check_audio_file(file_path):
    file_size = os.path.getsize(file_path)
    print(f"File size: {file_size} bytes")
    audio = MP3(file_path)
    duration = audio.info.length
    print(f"Duration: {duration} seconds")
    audio_segment = AudioSegment.from_mp3(file_path)
    average_dbfs = audio_segment.dBFS
    print(f"Average audio level: {average_dbfs} dBFS")
    return file_size, duration, average_dbfs

try:
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname, port, username, password)
    sftp = client.open_sftp()


    with open('/Users/tejeshchintpalli/Downloads/capstone/fulllogs1.txt', 'r') as file:
        urls = file.readlines()

    for url in urls:
        url = url.strip()  
        file_prefix = extract_file_prefix(url)
        file_suffix = first_suffix_for_sound_directory(url)
        updated_remote_directory = remote_directory + file_suffix + '/'

        files = sftp.listdir(updated_remote_directory)
        matching_files = [f for f in files if f.startswith(file_prefix)]

        img_filename = url.split('/')[-1]
        remote_img_path = url
        local_img_path = os.path.join(local_image_dir, img_filename)
        sftp.get(remote_img_path, local_img_path)
        print(f"Image file downloaded successfully to {local_img_path}")

        if not matching_files:
            print("No files found with the specified prefix")
        else:
            for file in matching_files:
                remote_path = os.path.join(updated_remote_directory, file)
                local_path = os.path.join(local_audio_dir if file.endswith('.mp3') else local_image_dir, file)
                sftp.get(remote_path, local_path)
                print(f"File downloaded successfully to {local_path}")

                if file.endswith('.mp3'):
                    file_size, duration, average_dbfs = check_audio_file(local_path)
                    if file_size > 0 and average_dbfs > -60:
                        print({"code": 0, "msg": "ok"})
                    else:
                        print({"code": 1, "msg": "Check Sound "})

    sftp.close()
    client.close()

except Exception as e:
    print(f"An error occurred: {e}")


Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimages/070035_15536.jpg
File downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaudio/070035_200000_.mp3
File size: 41779 bytes
Duration: 5.16 seconds
Average audio level: -22.3606278521674 dBFS
{'code': 0, 'msg': 'ok'}
Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimages/070035_15537.jpg
File downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaudio/070035_898000_.mp3
File size: 43124 bytes
Duration: 5.328 seconds
Average audio level: -22.53631931215857 dBFS
{'code': 0, 'msg': 'ok'}
Image file downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultimages/070050_15539.jpg
File downloaded successfully to /Users/tejeshchintpalli/Downloads/capstone/faultaudio/070050_300000_.mp3
File size: 41779 bytes
Duration: 5.16 seconds
Average audio level: -21.221249356255253 dBFS
{'code': 0, 'msg': 'ok'}
Image 

In [3]:
import paramiko
from mutagen.mp3 import MP3
from pydub import AudioSegment
import os
import pandas as pd

def extract_file_prefix(url):
    filename = url.split('/')[-1]
    return filename.split('_')[0]

def first_suffix_for_sound_directory(url):
    filename = url.split('/')[-2]
    return filename

hostname = 'files.clarksonmsda.org'
port = 2022
username = 'kadaliu@clarkson.edu'
password = 'Act.324fgdg'
remote_directory = '/srv/sftpgo/data/users/kadaliu@clarkson.edu/traffic/snd/'
local_image_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultimages/'
local_audio_dir = '/Users/tejeshchintpalli/Downloads/capstone/faultaudio/'

def check_audio_file(file_path):
    file_size = os.path.getsize(file_path)
    audio = MP3(file_path)
    duration = audio.info.length
    audio_segment = AudioSegment.from_mp3(file_path)
    average_dbfs = audio_segment.dBFS
    return file_size, duration, average_dbfs

# Dataframe to hold the log data
data_log = []

try:
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname, port, username, password)
    sftp = client.open_sftp()

    with open('/Users/tejeshchintpalli/Downloads/capstone/fulllogs.txt', 'r') as file:
        urls = file.readlines()

    for url in urls:
        url = url.strip()  # Clean up any whitespace
        file_prefix = extract_file_prefix(url)
        file_suffix = first_suffix_for_sound_directory(url)
        updated_remote_directory = remote_directory + file_suffix + '/'

        files = sftp.listdir(updated_remote_directory)
        matching_files = [f for f in files if f.startswith(file_prefix)]

        img_filename = url.split('/')[-1]
        remote_img_path = url
        local_img_path = os.path.join(local_image_dir, img_filename)
        sftp.get(remote_img_path, local_img_path)

        entry = {
            "URL": url,
            "File Prefix": file_prefix,
            "Remote Directory": updated_remote_directory,
            "Local Image Path": local_img_path,
            "Download Status": "Success"
        }

        if not matching_files:
            entry["Audio File Download Status"] = "No files found with the specified prefix"
        else:
            for file in matching_files:
                remote_path = os.path.join(updated_remote_directory, file)
                local_path = os.path.join(local_audio_dir if file.endswith('.mp3') else local_image_dir, file)
                sftp.get(remote_path, local_path)

                if file.endswith('.mp3'):
                    file_size, duration, average_dbfs = check_audio_file(local_path)
                    audio_entry = {
                        "File Size (bytes)": file_size,
                        "Duration (seconds)": duration,
                        "Average dBFS": average_dbfs
                    }
                    entry.update(audio_entry)
                    entry["Audio File Download Status"] = "Success" if file_size > 0 and average_dbfs > -60 else "Check Sound"

        data_log.append(entry)

    sftp.close()
    client.close()


    df = pd.DataFrame(data_log)
    df.to_excel('download_log.xlsx', index=False)

except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: Server connection dropped: 
